# DSC200 Lecture 9

## An introduction to NumPy (cont.)

## Recap

### Slices

Ellipsis can be specified explicitly with ```"..."```. It will automatically expand to `":"` for each of the unspecified dimensions in the array, and can even be used at the beginning of the slice:

In [1]:
import numpy as np
arr1 = np.empty((4, 6, 3))
print('Orig shape: ', arr1.shape)
print(arr1[...].shape)
print(arr1[..., 0:2].shape)
print(arr1[2:4, ..., ::2].shape)
print(arr1[2:4, :, ..., ::-1].shape)

Orig shape:  (4, 6, 3)
(4, 6, 3)
(4, 6, 2)
(2, 6, 2)
(2, 6, 3)


## Operating with arrays

It is important to remember that in general, all operations with arrays are applied *element-wise*, i.e., are applied to all the elements of the array at the same time.  For example:

In [2]:
arr1 = np.arange(4)
arr2 = np.arange(10, 14)
print(arr1, '+', arr2, '=', arr1 + arr2)

[0 1 2 3] + [10 11 12 13] = [10 12 14 16]


Importantly, even the multiplication operator is by default applied element-wise: It is *not* the matrix multiplication from linear algebra:

In [3]:
print(arr1, '*', arr2, '=', arr1 * arr2)

[0 1 2 3] * [10 11 12 13] = [ 0 11 24 39]


### Broadcasting

The fact that NumPy operates on an element-wise basis means that in principle arrays must always match one another's shape. However, NumPy will also helpfully "*broadcast*" dimensions when possible.  

Here is an example of broadcasting a scalar to a 1D array:

In [4]:
print(np.arange(3))
print(np.arange(3) + 5)

[0 1 2]
[5 6 7]


We can also broadcast a 1D array to a 2D array, in this case adding a vector to all rows of a matrix:

In [5]:
 np.ones((3, 3)) + np.arange(3) 

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

We can also broadcast in two directions at a time:

In [6]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)
print(a, '+', b, '=\n', a + b)

[[0]
 [1]
 [2]] + [0 1 2] =
 [[0 1 2]
 [1 2 3]
 [2 3 4]]


Pictorially:

![Illustration of broadcasting](./images/fig_broadcast_visual_1.png)

([image source](http://www.astroml.org/book_figures/appendix/fig_broadcast_visual.html))

## Reshape and newaxis

Reshaping arrays is a common task in order to make the best of NumPy's powerful broadcasting.

A useful tip with the **``reshape``** method is that it is possible to provide a ``-1`` length for at most one of the dimensions. This indicates that NumPy should automatically calculate the length of this dimension:

In [7]:
np.arange(6).reshape((1, -1))

array([[0, 1, 2, 3, 4, 5]])

In [8]:
np.arange(6).reshape((2, -1))

array([[0, 1, 2],
       [3, 4, 5]])

Another way to increase the dimensionality of an array is to use the ``newaxis`` keyword:

In [9]:
arr = np.arange(6)
print(arr[np.newaxis, :, np.newaxis].shape)

(1, 6, 1)


### Questions:

What will the result of adding ``arr1`` with ``arr2`` be in the following cases?

In [10]:
arr1 = np.ones((1, 3))
arr2 = np.ones((2, 1))

arr1 + arr2

array([[2., 2., 2.],
       [2., 2., 2.]])

In [11]:
arr1 = np.ones((1, 3))
arr2 = np.ones((1, 2))

arr1 + arr2

ValueError: operands could not be broadcast together with shapes (1,3) (1,2) 

In [16]:
arr1 = np.ones((1, 3))
arr2 = np.ones((1, 2))
arr3 = arr2[:, :, np.newaxis]

(arr1 + arr3).shape

(1, 2, 3)

## Array Properties and Methods (cont.)

For multidimensional arrays it is possible to carry out computations along a single dimension by passing the `axis` parameter:

In [19]:
arr = np.arange(6).reshape((2, -1))
print('For the following array:\n', arr)
print('The sum of elements along the rows is    :', arr.sum(axis=1))
print('The sum of elements along the columns is :', arr.sum(axis=0))

For the following array:
 [[0 1 2]
 [3 4 5]]
The sum of elements along the rows is    : [ 3 12]
The sum of elements along the columns is : [3 5 7]


As you can see in this example, the value of the `axis` parameter is the dimension that will be *consumed* once the operation has been carried out.  This is why to sum along the columns we use `axis=0`.  

This can be easily illustrated with an example that has more dimensions; we create an array with 4 dimensions and shape `(3,4,5,6)` and sum along the axis number 2 (i.e. the *third* axis, since in Python all counts are 0-based).  That consumes the dimension whose length was 5, leaving us with a new array that has shape `(3,4,6)`:

In [24]:
np.zeros((3, 4, 5, 6)).sum(axis=0).shape

(4, 5, 6)

Another widely used property of arrays is the `.T` attribute, which allows you to access the transpose of the array:

In [21]:
print('Array:\n', arr)
print('Transpose:\n', arr.T)

Array:
 [[0 1 2]
 [3 4 5]]
Transpose:
 [[0 3]
 [1 4]
 [2 5]]


## Generating 2D coordinate arrays

A common task is to generate a pair of arrays that represent the coordinates of our data.

When orthogonal 1d coordinate arrays already exist, NumPy's `meshgrid` function is very useful:

In [ ]:
x = np.linspace(0, 9, 3)
y = np.linspace(-8, 4, 3)
x2d, y2d = np.meshgrid(x, y)
print(x2d)
print(y2d)

### Views, not Copies

Note that reshaping (like most NumPy operations), wherever possible, provides a **view** of *the same memory*:

In [25]:
arr = np.arange(8)
arr_view = arr.reshape(2, 4)

What this means is that if one array is modified, the other will also be updated:

In [26]:
# Print the "view" array from reshape.
print('Before\n', arr_view)

# Update the first element of the original array.
arr[0] = 1000

# Print the "view" array from reshape again,
# noticing the first value has changed.
print('After\n', arr_view)

Before
 [[0 1 2 3]
 [4 5 6 7]]
After
 [[1000    1    2    3]
 [   4    5    6    7]]


This lack of copying allows for very efficient vectorized operations, but this power should be used carefully - if used badly it can lead to some bugs that are hard to track down.

If in doubt, you can always copy the data to a different block of memory with the **``copy()``** method.

## Element-wise Functions

NumPy ships with a full complement of mathematical functions that work on entire arrays, including logarithms, exponentials, trigonometric and hyperbolic trigonometric functions, etc.  

For example, sampling the sine function at 100 points between $0$ and $2\pi$ is as simple as:

In [29]:
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

Or to sample the exponential function between $-5$ and $5$ at intervals of $0.5$:

In [30]:
x = np.arange(-5, 5.5, 0.5)
y = np.exp(x)

## Linear algebra in NumPy

NumPy ships with a basic linear algebra library, and all arrays have a `dot` method whose behavior is that of the scalar dot product when its arguments are vectors (one-dimensional arrays) and the traditional matrix multiplication when one or both of its arguments are two-dimensional arrays:

In [31]:
v1 = np.array([2, 3, 4])
v2 = np.array([1, 0, 1])

print(v1, '.', v2, '=', np.dot(v1, v2))

[2 3 4] . [1 0 1] = 6


For matrix-matrix multiplication, the regular $matrix \times matrix$ rules must be satisfied. For example $A \times A^T$:

In [33]:
A = np.arange(6).reshape(2, 3)
print(A, '\n')
print(np.dot(A.T, A))

[[0 1 2]
 [3 4 5]] 

[[ 9 12 15]
 [12 17 22]
 [15 22 29]]


results in a (2, 2) array, yet $A^T \times A$ results in a (3, 3). 

Why is this?:

In [34]:
print(np.dot(A.T, A))

[[ 9 12 15]
 [12 17 22]
 [15 22 29]]


NumPy makes no distinction between row and column vectors and simply verifies that the dimensions match the required rules of matrix multiplication. 

Below is an example of matrix-vector multiplication, and in this case we have a $2 \times 3$ matrix multiplied by a 3-vector, which produces a 2-vector:

In [35]:
print(A, 'x', v1, '=', np.dot(A, v1))

[[0 1 2]
 [3 4 5]] x [2 3 4] = [11 38]


Note: To help with the interpretation of this last result, notice that $0 \times 2 + 1 \times 3 + 2 \times 4 = 11$ and $3 \times 2 + 4 \times 3 + 5 \times 4 = 38$